<a href="https://colab.research.google.com/github/divyaa-vermaa03/Practice_codes/blob/main/practice_code3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle tensorflow numpy opencv-python scikit-learn matplotlib

In [2]:
import os

# set kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
os.environ['KAGGLE_KEY'] = 'your_kaggle_api_key'

# Download the dataset from kaggle
! kaggle datasets download -d paultimothymooney/breast-histopathology-images

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images
License(s): CC0-1.0
100% 3.10G/3.10G [00:36<00:00, 128MB/s]
100% 3.10G/3.10G [00:36<00:00, 91.4MB/s]


In [3]:
import zipfile

with zipfile.ZipFile('breast-histopathology-images.zip', 'r') as zip_ref:
    zip_ref.extractall('Breast-histopathology-images')

In [6]:
import os
import cv2

def extract_images(dataset_path, output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  for root, _,files in os.walk(dataset_path):
      for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
          image_path = os.path.join(root, file)
          try:
            image = cv2.imread(image_path)
            if image is not None:
              relative_path = os.path.relpath(root, dataset_path)
              output_subdir = os.path.join(output_dir, relative_path)
              os.makedirs(output_subdir, exist_ok=True)
              output_path = os.path.join(output_subdir, file)
              cv2.imwrite(output_path, image)
              print(f"Extracted: {image_path} to {output_path}")
            else:
              print(f"Error reading image: {image_path}")
          except Exception as e:
            print(f"Error processing image: {image_path} - {e}")


dataset_path = 'Breast-histopathology-images/IDC_regular_ps50_idx5'
output_dir = 'extracted_images'

extract_images(dataset_path, output_dir)



Streaming output truncated to the last 5000 lines.
Extracted: Breast-histopathology-images/IDC_regular_ps50_idx5/14079/0/14079_idx5_x2151_y351_class0.png to extracted_images/14079/0/14079_idx5_x2151_y351_class0.png
Extracted: Breast-histopathology-images/IDC_regular_ps50_idx5/14079/0/14079_idx5_x2951_y801_class0.png to extracted_images/14079/0/14079_idx5_x2951_y801_class0.png
Extracted: Breast-histopathology-images/IDC_regular_ps50_idx5/14079/0/14079_idx5_x1401_y1251_class0.png to extracted_images/14079/0/14079_idx5_x1401_y1251_class0.png
Extracted: Breast-histopathology-images/IDC_regular_ps50_idx5/14079/0/14079_idx5_x2251_y1301_class0.png to extracted_images/14079/0/14079_idx5_x2251_y1301_class0.png
Extracted: Breast-histopathology-images/IDC_regular_ps50_idx5/14079/0/14079_idx5_x1901_y1351_class0.png to extracted_images/14079/0/14079_idx5_x1901_y1351_class0.png
Extracted: Breast-histopathology-images/IDC_regular_ps50_idx5/14079/0/14079_idx5_x1851_y1051_class0.png to extracted_images

In [7]:
import os

dataset_path = "extracted_images"
print("Classes found:", os.listdir(dataset_path))

Classes found: ['14191', '10295', '12751', '10300', '12824', '12911', '9083', '9037', '9123', '13461', '12910', '12929', '9228', '8975', '13025', '15840', '9265', '9381', '10304', '10277', '14209', '13404', '15516', '8950', '15903', '16167', '13401', '15510', '15513', '10307', '9345', '9125', '9254', '12811', '14157', '10299', '12868', '10306', '13022', '16568', '8951', '9346', '12750', '10285', '16555', '13688', '14154', '13021', '12900', '12908', '12821', '9382', '9077', '9175', '13693', '9250', '14190', '9044', '8916', '9347', '10286', '10259', '14304', '16551', '8956', '9041', '12818', '9258', '8917', '15514', '10272', '9262', '16165', '8864', '12907', '9325', '12626', '14153', '15512', '10269', '9078', '10290', '14081', '9173', '10291', '13692', '12906', '9267', '10264', '9266', '14213', '12909', '12752', '12876', '9255', '9081', '14078', '8984', '12933', '16533', '12869', '9261', '12935', '13616', '12749', '16166', '9256', '9291', '10293', '12905', '12892', '14305', '13020', '140

In [8]:
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

IMG_SIZE = 128
BATCH_SIZE = 32  # Load images in batches to save memory

def preprocess_image(image_path):
    """Load, resize, and normalize an image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read in grayscale
    if img is not None:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize
        img = img / 255.0  # Normalize
        img = np.expand_dims(img, axis=-1)  # Expand dims for TensorFlow
        return img
    return None

def load_image_paths(dataset_path):
    """Load image paths and labels dynamically without storing full images in RAM."""
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(dataset_path))
    class_map = {name: i for i, name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            for root, _, files in os.walk(class_dir):
                for file in files:
                    if file.lower().endswith((".png", ".jpg", ".jpeg")):  # Ensure valid image files
                        file_path = os.path.join(root, file)
                        image_paths.append(file_path)
                        labels.append(class_map[class_name])

    if not image_paths:
        print("A ⚠️ No images found! Ensure dataset is correctly loaded.")
        return None, None, None

    return image_paths, np.array(labels), class_names

# Load image paths and labels
dataset_path = "extracted_images"
image_paths, labels, class_names = load_image_paths(dataset_path)

if image_paths is not None and labels is not None:
    # Split data into training and testing sets
    X_train_paths, X_test_paths, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

    print(f"✅ Dataset Loaded: {len(X_train_paths)} training images, {len(X_test_paths)} testing images")

    # Data Generator to load images dynamically
    def image_data_generator(image_paths, labels, batch_size):
        while True:
            for i in range(0, len(image_paths), batch_size):
                batch_paths = image_paths[i:i+batch_size]
                batch_labels = labels[i:i+batch_size]

                images = [preprocess_image(path) for path in batch_paths if preprocess_image(path) is not None]
                labels_onehot = to_categorical(batch_labels, num_classes=len(class_names))

                if len(images) > 0:
                    yield np.array(images), np.array(labels_onehot)

    train_generator = image_data_generator(X_train_paths, y_train, BATCH_SIZE)
    test_generator = image_data_generator(X_test_paths, y_test, BATCH_SIZE)

else:
    print("❌ Dataset loading failed.")

✅ Dataset Loaded: 222019 training images, 55505 testing images


In [13]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ✅ Reduce TensorFlow logging (optional for cleaner output)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ✅ Check for GPU acceleration
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU detected! Training will be faster.")
else:
    print("⚠️ No GPU found. Training will use CPU.")

# ✅ Define Dataset Path
DATASET_PATH = "extracted_images"

# ✅ Image and training parameters (optimized for CPU)
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 5

# ✅ Data Augmentation (disabled for faster CPU training)
train_datagen = ImageDataGenerator(rescale=1.0/255.0,validation_split=0.2)

# ✅ Train & Validation Data Generators
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation')

# ✅ Get number of classes
num_classes = len(train_generator.class_indices)

# ✅ Define Lightweight CNN Model (from your 5th image)
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2, 2),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')])

# ✅ Compile the Model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# ✅ Callbacks
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# ✅ Train the Model (use limited steps per epoch for speed)
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=20,
    callbacks=[checkpoint, early_stopping]
)

# ✅ Save Final Model
model.save("final_model.h5")
print("🎉 Model training completed and saved successfully!")

⚠️ No GPU found. Training will use CPU.
Found 222127 images belonging to 279 classes.
Found 55397 images belonging to 279 classes.
Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.0061 - loss: 5.6160

100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 115ms/step - accuracy: 0.0061 - loss: 5.6159 - val_accuracy: 0.0047 - val_loss: 5.5656
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0081 - loss: 5.5539

100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.0081 - loss: 5.5538 - val_accuracy: 0.0063 - val_loss: 5.5091
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0093 - loss: 5.5133

100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.0093 - loss: 5.5133 - val_accuracy: 0.0094 - val_loss: 5.4892
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 105ms/step - accuracy: 0.0045 - loss: 5.5035 - val_accuracy: 0.0078 - val_loss: 5.4960
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.0100 - loss: 5.4909

100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.0100 - loss: 5.4908 - val_accuracy: 0.0109 - val_loss: 5.4543


🎉 Model training completed and saved successfully!


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')  # Multi-class classification
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create the model
# Assuming IMG_SIZE and class_names are defined earlier in your code
input_shape = (IMG_SIZE, IMG_SIZE, 1) # Grayscale images
num_classes = len(class_names)
model = create_cnn_model(input_shape, num_classes)

# Print model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 279)            │        35,991 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,660,567 (6.33 MB)

 Trainable params: 1,660,567 (6.33 MB)

 Non-trainable params: 0 (0.00 B)